In [33]:
from pathlib import Path

import numpy as np

data_dir = Path("data_old")
data_dir.mkdir(parents=True, exist_ok=True)

rng_seed = 42

print(f"Data will be saved to {data_dir.resolve()}")
print(f"Random seed is {rng_seed}")

Data will be saved to /Users/Tonni/Desktop/master-code/neural-quantum-tomo/case_studies/w_phase_augmented/data
Random seed is 42


In [34]:
def measure_qubit(plus_slice, minus_slice, op, rng):

    if op not in ('I', 'Z', 'X', 'Y'):
        raise ValueError(f"Unsupported Pauli operator: {op}. Expected 'I', 'X', 'Y', or 'Z'.")

    if op == 'I':
        return 0

    INV_SQRT2 = 1.0 / np.sqrt(2.0)

    plus_slice_tmp = plus_slice.copy()
    minus_slice_tmp = minus_slice.copy()

    # initialize with the Z basis
    plus_ampl = plus_slice_tmp
    minus_ampl = minus_slice_tmp

    plus_eigvec = (1.0, 0.0)
    minus_eigvec = (0.0, 1.0)


    if op == 'X':
        plus_ampl = (plus_slice_tmp + minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp - minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, INV_SQRT2)
        minus_eigvec = (INV_SQRT2, -INV_SQRT2)


    if op == 'Y':
        plus_ampl = (plus_slice_tmp - 1j * minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp + 1j * minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, 1j * INV_SQRT2)
        minus_eigvec = (INV_SQRT2, -1j * INV_SQRT2)


    plus_prob = np.sum(np.abs(plus_ampl)**2)
    plus_prob = np.clip(plus_prob, 0.0, 1.0)
    minus_prob = 1.0 - plus_prob
    plus_meas = rng.random() < plus_prob


    chosen_ampl = (plus_ampl if plus_meas else minus_ampl)
    chosen_eigvec = (plus_eigvec if plus_meas else minus_eigvec)
    renorm_factor = np.sqrt(plus_prob if plus_meas else minus_prob)


    if renorm_factor < 1e-12:
        plus_slice[:] = 0.0
        minus_slice[:] = 0.0

        return 0 if plus_meas else 1


    plus_slice[:] = (chosen_ampl * chosen_eigvec[0]) / renorm_factor
    minus_slice[:] = (chosen_ampl * chosen_eigvec[1]) / renorm_factor
    return 0 if plus_meas else 1


def measure_state(state_vec, pauli_ops, rng):
    num_qubits = len(pauli_ops)

    state_tensor = state_vec.reshape((2,) * num_qubits)
    outcome_bits = []

    for qubit_idx, pauli_op in enumerate(pauli_ops):
        plus_slice  = state_tensor[(slice(None),) * qubit_idx + (0,)]
        minus_slice = state_tensor[(slice(None),) * qubit_idx + (1,)]

        outcome_bit = measure_qubit(plus_slice, minus_slice, pauli_op, rng)
        outcome_bits.append(outcome_bit)

    return outcome_bits


def sample_state(state_vec, pauli_ops, num_samples=1000, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    samples = []
    for _ in range(num_samples):
        state_vec_copy = state_vec.copy()
        outcome_bits = measure_state(state_vec_copy, pauli_ops, rng)
        samples.append(outcome_bits)

    return samples

In [35]:
def format_bytes(num_bytes):
    """Utility function to format bytes into KB, MB, GB, etc."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if num_bytes < 1024.0:
            return f"{num_bytes:.2f} {unit}"
        num_bytes /= 1024.0
    return f"{num_bytes:.2f} PB"


# construct the state

num_qubits = 20

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
w_aug = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    w_aug[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)


print(f"Size of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

for i in range(10):
    print(f"{i:0{num_qubits}b}: {w_aug[i]:.2f} + {w_aug[i].imag:.2f}j")

Size of state vector in memory: 16.00 MB 

00000000000000000000: 0.00+0.00j + 0.00j
00000000000000000001: -0.15-0.16j + -0.16j
00000000000000000010: 0.10-0.20j + -0.20j
00000000000000000011: 0.00+0.00j + 0.00j
00000000000000000100: 0.21+0.09j + 0.09j
00000000000000000101: 0.00+0.00j + 0.00j
00000000000000000110: 0.00+0.00j + 0.00j
00000000000000000111: 0.00+0.00j + 0.00j
00000000000000001000: -0.21-0.08j + -0.08j
00000000000000001001: 0.00+0.00j + 0.00j


In [36]:
measurement_dirs = []

# all Z
measurement_dirs.append(['Z'] * num_qubits)

# sliding XX window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'X'
    measurement_dirs.append(basis_list)

# sliding XY window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'Y'
    measurement_dirs.append(basis_list)


for i, basis in enumerate(measurement_dirs):
    print(f"Basis {i:2d}: {''.join(basis)}")

Basis  0: ZZZZZZZZZZZZZZZZZZZZ
Basis  1: XXZZZZZZZZZZZZZZZZZZ
Basis  2: ZXXZZZZZZZZZZZZZZZZZ
Basis  3: ZZXXZZZZZZZZZZZZZZZZ
Basis  4: ZZZXXZZZZZZZZZZZZZZZ
Basis  5: ZZZZXXZZZZZZZZZZZZZZ
Basis  6: ZZZZZXXZZZZZZZZZZZZZ
Basis  7: ZZZZZZXXZZZZZZZZZZZZ
Basis  8: ZZZZZZZXXZZZZZZZZZZZ
Basis  9: ZZZZZZZZXXZZZZZZZZZZ
Basis 10: ZZZZZZZZZXXZZZZZZZZZ
Basis 11: ZZZZZZZZZZXXZZZZZZZZ
Basis 12: ZZZZZZZZZZZXXZZZZZZZ
Basis 13: ZZZZZZZZZZZZXXZZZZZZ
Basis 14: ZZZZZZZZZZZZZXXZZZZZ
Basis 15: ZZZZZZZZZZZZZZXXZZZZ
Basis 16: ZZZZZZZZZZZZZZZXXZZZ
Basis 17: ZZZZZZZZZZZZZZZZXXZZ
Basis 18: ZZZZZZZZZZZZZZZZZXXZ
Basis 19: ZZZZZZZZZZZZZZZZZZXX
Basis 20: XYZZZZZZZZZZZZZZZZZZ
Basis 21: ZXYZZZZZZZZZZZZZZZZZ
Basis 22: ZZXYZZZZZZZZZZZZZZZZ
Basis 23: ZZZXYZZZZZZZZZZZZZZZ
Basis 24: ZZZZXYZZZZZZZZZZZZZZ
Basis 25: ZZZZZXYZZZZZZZZZZZZZ
Basis 26: ZZZZZZXYZZZZZZZZZZZZ
Basis 27: ZZZZZZZXYZZZZZZZZZZZ
Basis 28: ZZZZZZZZXYZZZZZZZZZZ
Basis 29: ZZZZZZZZZXYZZZZZZZZZ
Basis 30: ZZZZZZZZZZXYZZZZZZZZ
Basis 31: ZZZZZZZZZZZXYZZZZZZZ
Basis 32

In [37]:
def format_measurement(bitstring, basis):
    result = []
    for bit, op in zip(bitstring, basis):
        if bit == 0 or bit == '0':
            result.append(op.upper())
        elif bit == 1 or bit == '1':
            result.append(op.lower())
        else:
            result.append('?')
    return ''.join(result)


samples_per_basis = 100 # 6400 in paper


for _, pauli_ops in enumerate(measurement_dirs):
    pauli_dirs_str = ''.join(pauli_ops)
    filename = data_dir / f"w_aug_{pauli_dirs_str}_{samples_per_basis}.txt"

    print(f"Measuring in Basis: {pauli_dirs_str}")

    samples = sample_state(w_aug, pauli_ops, samples_per_basis, rng=rng)

    with open(filename, 'w') as f_out:
        for bitstring in samples:
            formatted = format_measurement(bitstring, pauli_ops)
            f_out.write(formatted + "\n")

    print(f"Stored {samples_per_basis} samples to {filename.name}.")

Measuring in Basis: ZZZZZZZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZZZZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: XXZZZZZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_XXZZZZZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZXXZZZZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZXXZZZZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZXXZZZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZXXZZZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZXXZZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZXXZZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZZXXZZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZXXZZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZZZXXZZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZZXXZZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZZZZXXZZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZZZXXZZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZZZZZXXZZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZZZZXXZZZZZZZZZZZ_100.txt.
Measuring in Basis: ZZZZZZZZXXZZZZZZZZZZ
Stored 100 samples to w_aug_ZZZZZZZZXXZZZZZZZZZZ_100.txt.
Measuring 